<a href="https://colab.research.google.com/github/rorre/PyNotebooks/blob/master/osu!stuffs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# osu! ranked data


In [1]:
#@title Data Setup

#@markdown API key could be gathered [here](https://osu.ppy.sh/p/api)
api_key = "" #@param {type:"string"}

#@markdown Starting data date
starting_date = "2020-07-20" #@param {type:"date"}

#@markdown Game mode
gamemode = "osu!" #@param ["osu!", "osu!taiko", "osu!catch", "osu!mania"]

_dropdown_values = {
    "osu!": 0,
    "osu!taiko": 1,
    "osu!catch": 2,
    "osu!mania": 3
}
_mode = _dropdown_values.get(gamemode, 0)



## Fetch data

In [3]:
from pprint import pprint
from datetime import date
from dateutil.parser import parse

import requests

def find_earliest(beatmaps):
  earliest = date.fromtimestamp(0)
  for bm in beatmaps:
    ranked_date = bm['approved_date'].split(" ")[0]
    current = parse(ranked_date).date()
    if current > earliest:
      earliest = current
  
  return earliest.isoformat()


# Get all *ranked* beatmaps from starting point
# If request reaches limit, find the earliest beatmap in the result and start
# working again from that point up until response is less than the limit.
results = []
while True:
  r = requests.get("https://osu.ppy.sh/api/get_beatmaps?k={}&m={}&since={}".format(api_key, _mode, starting_date))
  r.raise_for_status()

  json_result = r.json()
  if not json_result:
    break

  filtered_result = filter(lambda x: x["approved"] == "1", json_result)
  results.extend(list(filtered_result))

  if len(json_result) == 500:
    starting_date = find_earliest(json_result)
  else:
    break

if not results:
  raise Exception("No data returned from osu!.")

# Only print one sample data
pprint(results[0])

{'approved': '1',
 'approved_date': '2020-07-20 03:45:44',
 'artist': 'Reol',
 'artist_unicode': 'れをる',
 'audio_unavailable': '0',
 'beatmap_id': '2234222',
 'beatmapset_id': '1067141',
 'bpm': '140',
 'count_normal': '450',
 'count_slider': '553',
 'count_spinner': '0',
 'creator': 'vikala',
 'creator_id': '2848604',
 'diff_aim': '2.69452',
 'diff_approach': '9.2',
 'diff_drain': '6',
 'diff_overall': '9',
 'diff_size': '4.2',
 'diff_speed': '2.56905',
 'difficultyrating': '5.32631',
 'download_unavailable': '0',
 'favourite_count': '135',
 'file_md5': 'a079e36c8a4a8abd4383d6168fbb91f4',
 'genre_id': '5',
 'hit_length': '231',
 'language_id': '3',
 'last_update': '2020-07-13 00:09:50',
 'max_combo': '1613',
 'mode': '0',
 'packs': 'S911',
 'passcount': '1233',
 'playcount': '14562',
 'rating': '8.86335',
 'source': '',
 'storyboard': '0',
 'submit_date': '2019-11-18 01:31:37',
 'tags': 'firika ioexception deadcode toybot giga okiku no title+ vocaloid '
         'hatsune miku 初音ミク japa

## Populate data

In [4]:
# Populate data to a list.
# Track for the maps that have been checked so it doesn't have duplicates.
beatmaps = []
checked_maps = []
for beatmap in results:
  mapset_id = beatmap['beatmapset_id']
  if mapset_id in checked_maps:
    continue
  checked_maps.append(mapset_id)
  beatmaps.append(beatmap)

## osu! ranked/day

In [5]:
from collections import Counter

data_list = []
for beatmap in beatmaps:
  ranked_date = beatmap['approved_date'].split(" ")[0]
  data_list.append(ranked_date)

c = Counter(data_list)
for k, v in c.items():
  print(f"{k}: {v}")

2020-07-20: 5
2020-07-21: 4
2020-07-22: 7
2020-07-23: 5
2020-07-24: 4
2020-07-25: 4
2020-07-26: 7
2020-07-27: 3
2020-07-28: 1
2020-07-29: 6
2020-07-30: 8
2020-07-31: 3
2020-08-01: 8
2020-08-02: 7
2020-08-03: 2


### Plot data

In [6]:
import plotly.graph_objects as go

x = []
y = []
for k, v in c.items():
  x.append(k)
  y.append(v)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode='lines'))
fig.show()

## osu! mapper

In [7]:
from collections import Counter

mappers = []
for beatmap in beatmaps:
  mappers.append(beatmap['creator'])

c = Counter(mappers)
for k, v in c.items():
  print(f"{k}: {v}")

vikala: 1
Petal: 2
Mirash: 1
- ascended -: 2
ShirohaMyMommy: 1
-Aqua: 1
Sotarks: 3
Rosiie: 1
Smoke: 1
Chaoslitz: 1
Sylvarus: 1
Nevo: 2
Zer0-: 1
Sarawatlism: 1
SMOKELIND: 2
lewski: 1
[-Evil-]: 1
Yumerios: 1
Elinor: 1
Blitzifyyy: 1
emu1337: 1
Stack: 1
domSaur: 1
Silent Rock: 1
Doormat: 1
Parachute: 1
Krisom: 1
Heilia: 1
Yohanes: 1
X Light: 1
Seolv: 1
Agatsu: 2
Setu: 1
Moko: 1
UberFazz: 1
Fall: 2
-Atri-: 1
Xandit: 1
Lunala: 1
-Mikan: 1
Mordred: 1
Crissa: 2
Hachiko: 1
Akino Hana: 1
Daycore: 1
papple104: 1
Toyosaki Aki: 1
wafer: 1
Amateurre: 1
DuckRadio: 1
AirinCat: 1
Frostings: 1
Sephira: 1
dkblaze: 1
browiec: 1
SaltyLucario: 1
Luscent: 1
Hey lululu: 1
Akito: 1
Melwoine: 1
Addy2607: 1
HeTo: 1
SuperDalouBot: 1
Roger: 1
Cellina: 1


### Plot data

In [8]:
import plotly.graph_objects as go
import numpy as np

def random_rgb():
  colors_int = list(np.random.choice(range(256), size=3))
  colors = list(map(str, colors_int))
  return f"rgb({', '.join(colors)})"

def calculate_size(size, max):
  return 200 * (size / max)

x = []
y = []
for k, v in c.most_common():
  x.append(k)
  y.append(v)
max_y = max(y)

bars = go.Bar(x=x, y=y)
fig = go.Figure(data=[bars])
fig.show()

## Table leaderboard

In [9]:
!pip install tabletext

  Created wheel for tabletext: filename=tabletext-0.1-cp36-none-any.whl size=6023 sha256=63826c134149e40223a2f4b5b6eadbf700cad3628b9e4bdcd165e585683eae0d
  Stored in directory: /root/.cache/pip/wheels/63/15/d8/897b137f43975c4f5f49139be65fee6dbeab6a3f88c1838f66
Successfully built tabletext


In [11]:
import tabletext
print(tabletext.to_text(c.most_common()))

┌────────────────┬───┐
│ Sotarks        │ 3 │
├────────────────┼───┤
│ Petal          │ 2 │
├────────────────┼───┤
│ - ascended -   │ 2 │
├────────────────┼───┤
│ Nevo           │ 2 │
├────────────────┼───┤
│ SMOKELIND      │ 2 │
├────────────────┼───┤
│ Agatsu         │ 2 │
├────────────────┼───┤
│ Fall           │ 2 │
├────────────────┼───┤
│ Crissa         │ 2 │
├────────────────┼───┤
│ vikala         │ 1 │
├────────────────┼───┤
│ Mirash         │ 1 │
├────────────────┼───┤
│ ShirohaMyMommy │ 1 │
├────────────────┼───┤
│ -Aqua          │ 1 │
├────────────────┼───┤
│ Rosiie         │ 1 │
├────────────────┼───┤
│ Smoke          │ 1 │
├────────────────┼───┤
│ Chaoslitz      │ 1 │
├────────────────┼───┤
│ Sylvarus       │ 1 │
├────────────────┼───┤
│ Zer0-          │ 1 │
├────────────────┼───┤
│ Sarawatlism    │ 1 │
├────────────────┼───┤
│ lewski         │ 1 │
├────────────────┼───┤
│ [-Evil-]       │ 1 │
├────────────────┼───┤
│ Yumerios       │ 1 │
├────────────────┼───┤
│ Elinor   